In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import os
from dotenv import load_dotenv

In [2]:
from sqlalchemy import create_engine, text, Column, Integer, String, MetaData, Table, BigInteger, Float, VARCHAR, TEXT, BIGINT
from sqlalchemy.orm import declarative_base

In [3]:
load_dotenv()

True

In [4]:
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")

In [5]:
database = pd.read_csv('TMDB_movie_dataset_cleaned.csv')

In [ ]:
# database.loc[564495,:]

vote_average                   0.0
vote_count                       0
release_date                  2024
revenue                          0
runtime                         28
adult                        False
budget                         291
original_language               en
original_title          Manchester
popularity                     1.4
genres                         NaN
production_companies           NaN
production_countries        Brazil
spoken_languages        Portuguese
keywords                       NaN
Name: 564495, dtype: object

In [6]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}')

In [7]:
with engine.connect() as conn:
        conn.execute(text("CREATE DATABASE IF NOT EXISTS movies_db"))
        print("Database created succesfully")

Database created succesfully


In [8]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018562 entries, 0 to 1018561
Data columns (total 15 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   vote_average          1018562 non-null  float64
 1   vote_count            1018562 non-null  int64  
 2   release_date          1018562 non-null  int64  
 3   revenue               1018562 non-null  int64  
 4   runtime               1018562 non-null  int64  
 5   adult                 1018562 non-null  bool   
 6   budget                1018562 non-null  int64  
 7   original_language     1018562 non-null  object 
 8   original_title        1018561 non-null  object 
 9   popularity            1018562 non-null  float64
 10  genres                677660 non-null   object 
 11  production_companies  526683 non-null   object 
 12  production_countries  643007 non-null   object 
 13  spoken_languages      647296 non-null   object 
 14  keywords              305240 non-n

In [9]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/movies_db')
metadata = MetaData()

In [10]:
table = 'movies'

In [18]:
columns = [Column('id', Integer, primary_key=True, autoincrement=True)]

for col_name in database.columns:
 
    if database[col_name].dtype == 'int64':
        columns.append(Column(col_name, BIGINT))
    else:
        columns.append(Column(col_name, TEXT))

movies_table = Table('movies', metadata, *columns)

metadata.create_all(engine)
print("Created Table ✅")

Created Table ✅


In [11]:
def select_first_records(table, records=5):
    with engine.connect() as conn:
        result = conn.execute(text(f"""
                                
                                    SELECT * 
                                    FROM {table}
                                    LIMIT {records}

                                    """))
    return result.fetchall()

In [12]:
def select_first_records_columns(table, columns, records=5, order='id'):
    with engine.connect() as conn:
        result = conn.execute(text(f"""
                                
                                    SELECT {', '.join(columns)} 
                                    FROM {table}
                                    ORDER BY {order}
                                    LIMIT {records}
                                    
                                    """))
    return result.fetchall()

In [13]:
def delete_register(table, id):
    with engine.connect() as conn:
        conn.execute(text(f"DELETE FROM {table} WHERE id = {id}"))
        conn.commit()
        print(f"Register with id {id} deleted successfully ✅")

In [14]:
def delete_table(table):
    with engine.connect() as conn:
        conn.execute(text(f"DROP TABLE IF EXISTS {table}"))
        conn.commit()
        print(f"Table {table} deleted successfully ✅")

In [15]:
def insert_data(table, data, eng, chunksize=10000):
    data.to_sql(
        name=table,
        con=eng, 
        if_exists='append',
        index=False,
        chunksize=chunksize)

    print("Data loaded from dataframe csv successfuly ✅")

In [17]:
delete_table(table)

Table movies deleted successfully ✅


In [23]:
select_first_records(table, 12)

[(1, '8.364', 34495, 2010, 825532764, 148, '0', 160000000, 'en', 'Inception', '83.952', 'Action Science Fiction Adventure', 'Legendary Pictures Syncopy Warner Bros Pictures', 'United Kingdom, United States of America', 'English, French, Japanese, Swahili', 'rescue mission dream airplane paris france virtual reality kidnapping philosophy spy allegory manipulation car crash heist memory architecture los angeles california dream world subconscious'),
 (2, '8.417', 32571, 2014, 701729206, 169, '0', 165000000, 'en', 'Interstellar', '140.241', 'Adventure Drama Science Fiction', 'Legendary Pictures Syncopy Lynda Obst Productions', 'United Kingdom, United States of America', 'English', 'rescue future spacecraft race against time artificial intelligence ai nasa time warp dystopia expedition space travel wormhole famine black hole quan ... (92 characters truncated) ... e station curious space adventure time paradox thoughtful timemanipulation father daughter relationship 2060s cornfield time man

In [20]:
consult_columns = ['id', 'original_title', 'release_date', 'vote_average', 'budget','revenue']
# consult_columns = database.columns.tolist()

In [22]:
select_first_records_columns(table, consult_columns, 40, order = 'original_title')

[(93368, None, 2013, '6', 0, 0),
 (140928, ': A Question Mark', 2012, '5.4', 98990, 0),
 (953821, '::64', 2017, '0', 0, 0),
 (602310, ':03 from Gold', 2002, '0', 0, 0),
 (820556, ':another story:', 2021, '0', 0, 0),
 (837458, '&  fronds falling palmed off among frame', 2016, '0', 0, 0),
 (717953, '&  Là il cielo e la terra si univano là le stagioni si ricongiungevano là il vento e la pioggia si univano', 1972, '0', 0, 0),
 (597671, '& Etc', 2007, '0', 0, 0),
 (881427, '& More', 2022, '0', 0, 0),
 (538425, '& Nothing More', 2025, '0', 0, 0),
 (192930, '& Teller', 2008, '5.5', 0, 0),
 (242631, '& Teller 2', 2008, '6', 0, 0),
 (1009121, '& Teller 3', 2014, '0', 0, 0),
 (1009120, '& Teller 4', 2014, '0', 0, 0),
 (1009098, '& Teller 5', 2014, '0', 0, 0),
 (637155, '& जर हटक', 2016, '0', 0, 0),
 (502176, '&Human', 2011, '0', 0, 0),
 (133050, '&Me', 2013, '4.2', 0, 0),
 (922530, '# 32', 2018, '0', 0, 0),
 (985027, '#0 sisocisP', 1992, '0', 0, 0),
 (852704, '#001', 2019, '0', 0, 0),
 (852705, 

In [ ]:
# insert_data(table, database, engine)

Data loaded from dataframe csv successfuly ✅
